In [16]:
import pandas as pd
import os

# Define the directory containing Excel files
directory = 'report_subset'

# Initialize an empty list to store the consolidated data
consolidated_data = []

# Loop through all files in the directory
for filename in os.listdir(directory):
    if filename.endswith('.xlsx') or filename.endswith('.xls'):
        file_path = os.path.join(directory, filename)
        
        # Load the Excel file
        df = pd.read_excel(file_path, sheet_name='sacc_invoice', header=None)
        
        # Extract 'year and account info', 'invoiced month', and 'parent info'
        year_and_account_info = df.at[7, 0]
        invoiced_month = df.at[8, 0]
        parent_info = df.at[6, 3]
        
        # Initialize a dictionary to hold the row data
        row_data = {
            'year and account info': year_and_account_info,
            'invoiced month': invoiced_month,
            'parent info': parent_info
        }

        # Handle "Previous Activity"
        previous_activity_start = df[df[0] == 'Previous Activity'].index[0] if 'Previous Activity' in df[0].values else None
        
        if previous_activity_start is not None:
            i = previous_activity_start + 1
            previous_activity = []
            
            while i < len(df) and df.at[i, 0] != 'Start Date':
                previous_activity.append("@".join(str(cell) for cell in df.iloc[i].dropna()))
                i += 1
            
            row_data['previous activity'] = "@".join(previous_activity)

        # Handle the child-related data starting from "Start Date"
        start_row = df[df[0] == 'Start Date'].index[0] if 'Start Date' in df[0].values else None
        
        if start_row is not None:
            child_counter = 1
            i = start_row + 1  # Start looking at the row after "Start Date"
            
            while i < len(df):
                if 'Discounts and Adjustments' in str(df.at[i, 0]):
                    break
                
                child_name = df.at[i, 0]
                if i + 1 < len(df):
                    child_info = "@".join(str(cell) for cell in df.iloc[i + 1].dropna())
                else:
                    child_info = ""
                
                row_data[f'child_{child_counter}_name'] = child_name
                row_data[f'child_{child_counter}_info'] = child_info
                
                i += 2
                child_counter += 1
                if child_counter > 6:
                    break

        # Handle "Discounts and Adjustments"
        discounts_start = df[df[0] == 'Discounts and Adjustments'].index[0] if 'Discounts and Adjustments' in df[0].values else None
        
        if discounts_start is not None:
            i = discounts_start + 1
            discounts_adjustments = []
            
            while i < len(df) and 'Total Due' not in str(df.at[i, 0]):
                discounts_adjustments.append("@".join(str(cell) for cell in df.iloc[i].dropna()))
                i += 1
            
            row_data['discounts and adjustments'] = "@".join(discounts_adjustments)

            # Handle "Total Due" row
            if i < len(df):
                row_data['Total Due'] = "@".join(str(cell) for cell in df.iloc[i].dropna())
        
        # Handle "invoice #" (last row in the sheet)
        last_row = df.iloc[-1]
        row_data['invoice #'] = last_row.dropna().values[0]

        # Append the row data to the consolidated data list
        consolidated_data.append(row_data)

# Convert the list of dictionaries to a DataFrame
consolidated_df = pd.DataFrame(consolidated_data)

# Save the consolidated data to a new Excel file
consolidated_df.to_excel('consolidated_data.xlsx', index=False)


In [20]:
import pandas as pd
import os

# Define the directory containing Excel files
directory = 'report_subset'

# Initialize an empty list to store the consolidated data
consolidated_data = []

# Loop through all files in the directory
for filename in os.listdir(directory):
    if filename.endswith('.xlsx') or filename.endswith('.xls'):
        file_path = os.path.join(directory, filename)
        
        # Load the Excel file
        df = pd.read_excel(file_path, sheet_name='sacc_invoice', header=None)
        
        # Extract 'year and account info', 'invoiced month', and 'parent info'
        year_and_account_info = df.at[7, 0]
        invoiced_month = df.at[8, 0]
        parent_info = df.at[6, 3] if not pd.isna(df.at[6, 3]) else df.at[6, 3]  # Capture D7 value
        
        # Replace newlines with @ in the 'parent info'
        if isinstance(parent_info, str):
            parent_info = parent_info.replace('\n', '@')
        
        # Initialize a dictionary to hold the row data
        row_data = {
            'year and account info': year_and_account_info.replace('\n', '@') if isinstance(year_and_account_info, str) else year_and_account_info,
            'invoiced month': invoiced_month.replace('\n', '@') if isinstance(invoiced_month, str) else invoiced_month,
            'parent info': parent_info.replace('\n', '@') if isinstance(parent_info, str) else parent_info
        }

        # Handle "Previous Activity"
        previous_activity_start = df[df[0] == 'Previous Activity'].index[0] if 'Previous Activity' in df[0].values else None
        
        if previous_activity_start is not None:
            i = previous_activity_start + 1
            previous_activity = []
            
            while i < len(df) and df.at[i, 0] != 'Start Date':
                previous_activity.append("@".join(str(cell).replace('\n', '@') for cell in df.iloc[i].dropna()))
                i += 1
            
            row_data['previous activity'] = "@".join(previous_activity)

        # Handle the child-related data starting from "Start Date"
        start_row = df[df[0] == 'Start Date'].index[0] if 'Start Date' in df[0].values else None
        
        if start_row is not None:
            child_counter = 1
            i = start_row + 1  # Start looking at the row after "Start Date"
            
            while i < len(df):
                if 'Discounts and Adjustments' in str(df.at[i, 0]):
                    break
                
                child_name = df.at[i, 0].replace('\n', '@') if isinstance(df.at[i, 0], str) else df.at[i, 0]
                if i + 1 < len(df):
                    child_info = "@".join(str(cell).replace('\n', '@') for cell in df.iloc[i + 1].dropna())
                else:
                    child_info = ""
                
                row_data[f'child_{child_counter}_name'] = child_name
                row_data[f'child_{child_counter}_info'] = child_info
                
                i += 2
                child_counter += 1
                if child_counter > 6:
                    break

        # Handle "Discounts and Adjustments"
        discounts_start = df[df[0] == 'Discounts and Adjustments'].index[0] if 'Discounts and Adjustments' in df[0].values else None
        
        if discounts_start is not None:
            i = discounts_start + 1
            discounts_adjustments = []
            
            while i < len(df) and 'Total Due' not in str(df.at[i, 0]):
                discounts_adjustments.append("@".join(str(cell).replace('\n', '@') for cell in df.iloc[i].dropna()))
                i += 1
            
            row_data['discounts and adjustments'] = "@".join(discounts_adjustments)

            # Handle "Total Due" row
            if i < len(df):
                row_data['Total Due'] = "@".join(str(cell).replace('\n', '@') for cell in df.iloc[i].dropna())
        
        # Handle "invoice #" (last row in the sheet)
        last_row = df.iloc[-1]
        invoice_number = last_row.dropna().values[0] if len(last_row.dropna().values) > 0 else ""
        row_data['invoice #'] = invoice_number.replace('\n', '@') if isinstance(invoice_number, str) else invoice_number

        # Append the row data to the consolidated data list
        consolidated_data.append(row_data)

# Convert the list of dictionaries to a DataFrame
consolidated_df = pd.DataFrame(consolidated_data)

# Save the consolidated data to a new Excel file
consolidated_df.to_excel('consolidated_data2.xlsx', index=False)


In [1]:
import pandas as pd
import os

def concatenate_cells(cells):
    """Concatenate cells, replacing newlines with '@'."""
    return "@".join(str(cell).replace('\n', '@') for cell in cells if pd.notna(cell))

def load_excel_file(file_path):
    """Load the Excel file and return the DataFrame."""
    return pd.read_excel(file_path, sheet_name='sacc_invoice', header=None)

def extract_fields(df):
    """Extract 'year and account info', 'invoiced month', and 'parent info' from the DataFrame."""
    year_and_account_info = df.at[7, 0]
    invoiced_month = df.at[8, 0]
    parent_info = df.at[6, 3] if not pd.isna(df.at[6, 3]) else df.at[6, 3]  # Capture D7 value
    
    return {
        'year and account info': concatenate_cells([year_and_account_info]),
        'invoiced month': concatenate_cells([invoiced_month]),
        'parent info': concatenate_cells([parent_info])
    }

def handle_previous_activity(df):
    """Handle 'Previous Activity' section and return a dictionary with separate columns for each row."""
    previous_activity_start = df[df[0] == 'Previous Activity'].index[0] if 'Previous Activity' in df[0].values else None
    
    previous_activity_data = {}
    
    if previous_activity_start is not None:
        i = previous_activity_start + 1
        activity_counter = 1
        
        while i < len(df) and df.at[i, 0] != 'Start Date':
            # Create a unique column for each row
            column_name = f'previous activity {activity_counter}'
            previous_activity_data[column_name] = concatenate_cells(df.iloc[i])
            i += 1
            activity_counter += 1
    
    return previous_activity_data


def handle_children(df):
    """Handle child-related data and return a dictionary of child names and info."""
    start_row = df[df[0] == 'Start Date'].index[0] if 'Start Date' in df[0].values else None
    children_data = {}
    
    if start_row is not None:
        child_counter = 1
        i = start_row + 1
        
        while i < len(df):
            if 'Discounts and Adjustments' in str(df.at[i, 0]):
                break
            
            child_name = concatenate_cells([df.at[i, 0]])
            child_info = concatenate_cells(df.iloc[i + 1]) if i + 1 < len(df) else ""
            
            children_data[f'child_{child_counter}_name'] = child_name
            children_data[f'child_{child_counter}_info'] = child_info
            
            i += 2
            child_counter += 1
            if child_counter > 6:
                break
    
    return children_data

def handle_discounts_and_adjustments(df):
    """Handle 'Discounts and Adjustments' and 'Total Due' and return a dictionary with separate columns for each row."""
    discounts_start = df[df[0] == 'Discounts and Adjustments'].index[0] if 'Discounts and Adjustments' in df[0].values else None
    discounts_data = {}
    
    if discounts_start is not None:
        i = discounts_start + 1
        discounts_counter = 1
        discounts_adjustments = []
        
        while i < len(df) and 'Total Due' not in str(df.at[i, 0]):
            # Create a unique column for each row
            column_name = f'discounts and adjustments {discounts_counter}'
            discounts_data[column_name] = concatenate_cells(df.iloc[i])
            i += 1
            discounts_counter += 1
        
        # Handle 'Total Due' row
        if i < len(df):
            discounts_data['Total Due'] = concatenate_cells(df.iloc[i])
    
    return discounts_data


def handle_invoice_number(df):
    """Handle the last row for 'invoice #' and return the invoice number."""
    last_row = df.iloc[-1]
    invoice_number = last_row.dropna().values[0] if len(last_row.dropna().values) > 0 else ""
    return {'invoice #': concatenate_cells([invoice_number])}

def process_excel_file(file_path):
    """Process an Excel file and return a dictionary of extracted data."""
    df = load_excel_file(file_path)
    
    row_data = extract_fields(df)
    row_data.update(handle_previous_activity(df))
    row_data.update(handle_children(df))
    row_data.update(handle_discounts_and_adjustments(df))
    row_data.update(handle_invoice_number(df))
    
    return row_data

def main(directory):
    """Process all Excel files in the specified directory and save consolidated data."""
    consolidated_data = []

    for filename in os.listdir(directory):
        if filename.endswith('.xlsx') or filename.endswith('.xls'):
            file_path = os.path.join(directory, filename)
            row_data = process_excel_file(file_path)
            consolidated_data.append(row_data)

    # Convert the list of dictionaries to a DataFrame
    consolidated_df = pd.DataFrame(consolidated_data)

    # Save the consolidated data to a new Excel file
    consolidated_df.to_excel('consolidated_data2.xlsx', index=False)

if __name__ == "__main__":
    directory = 'reports_aug23'
    main(directory)
